# Importando os dados

In [1]:
import pandas as pd

In [2]:
dados_treino=pd.read_csv('/kaggle/input/alura-word2vec/treino.csv')

# Processamento de Linguagem Natural

In [ ]:
# Baixando as dependecias do spicy para pt-br
#!python -m spacy download pt_core_news_sm

In [3]:
import spacy
nlp=spacy.load('pt_core_news_sm') #modelo

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
texto="Rio de Janeiro é uma cidade maravilhosa"
nlp(texto)

Rio de Janeiro é uma cidade maravilhosa

In [ ]:
#Será que a saida é realmente um texto?
doc=nlp(texto)
type(doc)

# Não, ele é um objeto do tipo DOC com os dados pré-processados


In [ ]:
# Metodos 

#doc[0]
#doc.ents
#doc[0].is_stop
#doc[1].is_stop


# Tratando os textos

In [5]:
textos_para_tratamento=(titulos.lower() for titulos in dados_treino['title']) #generation expressions

In [ ]:
textos_para_tratamento #um generatorsó cria os itens quando forem necessários

In [6]:
def trata_textos(doc):
    tokens_validos=[]
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)
        
        

In [ ]:
#trata_textos(doc)
#[print (i) for i in textos_para_tratamento]

In [7]:
from time import time

t0=time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                    batch_size=1000,
                                                    n_process=-1)]

t1=time() -t0

print(t1/60)

2.9133864402770997


In [8]:
titulos_tratados = pd.DataFrame({
    'titulo':textos_tratados
})

titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


# Treinando o modelo - CBOW

In [9]:
from gensim.models import Word2Vec

w2v_modelo=Word2Vec(sg=0, #cbow =0,skipgram=1,
                   window=2, #palavras antes e depois,
                   vector_size=300,
                   min_count=5,
                   alpha=0.3,
                min_alpha=0.007#taxa de aprendizagem)
                   )

In [10]:
print(len(titulos_tratados))

titulos_tratados=titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))


90000
84466


In [11]:
lista_lista_tokens=[titulo.split(" ") for titulo in titulos_tratados.titulo]
w2v_modelo.build_vocab(lista_lista_tokens)
#Não aperece nenhuma mesagem

# Configurando as mensagens de log

In [ ]:
# Criando um vocabulario

import logging

logging.basicConfig(format="%(asctime)s : - %(message)s",level=logging.INFO)

w2v_modelo=Word2Vec(sg=0, #cbow =0,skipgram=1,
                   window=2, #palavras antes e depois,
                   vector_size=300,
                   min_count=5,
                   alpha=0.3,
                min_alpha=0.007#taxa de aprendizagem)
                   )
#acho q o kaggle nãp mostra
(w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000))

# Treinando o modelo

In [ ]:
dir(w2v_modelo)

In [12]:
# Numero de palvras no modelo
w2v_modelo.corpus_count

84466

In [13]:
w2v_modelo.train(corpus_iterable=lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                epochs=30)

(14584151, 16207260)

In [21]:
# Avaliando o modelo
w2v_modelo.wv.most_similar("lula")

[('reduza', 0.6501877307891846),
 ('ueba', 0.6305856704711914),
 ('rousseff', 0.6151136159896851),
 ('pedro', 0.6076457500457764),
 ('provocado', 0.5711051225662231),
 ('vestiu', 0.557428240776062),
 ('ativo', 0.5500892400741577),
 ('viaja', 0.5228115916252136),
 ('mandato', 0.5194615125656128),
 ('malhação', 0.5171372294425964)]

# Treinando o modelo - SKIPGRAM

In [22]:
w2v_modelo_sg=Word2Vec(sg=1, #cbow =0,skipgram=1,
                   window=5, #palavras antes e depois,
                   vector_size=300,
                   min_count=5,
                   alpha=0.3,
                min_alpha=0.007#taxa de aprendizagem)
                   )
#acho q o kaggle nãp mostra
(w2v_modelo_sg.build_vocab(lista_lista_tokens))

w2v_modelo_sg.train(corpus_iterable=lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                epochs=30)

(14584903, 16207260)

In [29]:
w2v_modelo_sg.wv.most_similar("c")

[('glauco', 0.988011360168457),
 ('bastilha', 0.9760313630104065),
 ('corajosa', 0.9687303900718689),
 ('facão', 0.9449235200881958),
 ('portabilidade', 0.9012660384178162),
 ('desconhece', 0.8830090165138245),
 ('ônibus', 0.8700390458106995),
 ('subsidiado', 0.8374739289283752),
 ('espera', 0.8363398313522339),
 ('leilões', 0.834123432636261)]

In [33]:
w2v_modelo_sg.wv.save_word2vec_format('/kaggle/working/modelo_skip_gram.txt',binary=False)

In [32]:
w2v_modelo.wv.save_word2vec_format('/kaggle/working/modelo_cbow.txt',binary=False)